## PDF loader

In [1]:
## Load pdf document

from langchain_community.document_loaders import PyPDFLoader

file_path = "D:\\GenAI\\Agentic AI\\Test_Projects\\RAG-Project\\data\\PDF\\Rainbow-Bazaar-Return-Refund-&-Cancellation-Policy.pdf"
pdf_loader = PyPDFLoader(file_path)

pdf_docs = pdf_loader.load()

d:\GenAI\Agentic AI\Test_Projects\RAG-Project\venv10\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Chunking

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter


In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [4]:
pdf_chunks = text_splitter.split_documents(pdf_docs)

In [5]:
print(len(pdf_chunks))

9


In [9]:
print(pdf_chunks[0].page_content[:1000])

Return, Refund and Cancellation Policy 
1.1. Refunds, Cancellations and Returns of Goods 
Please note that you can submit complaints with the Order and product sold via 
rainbowbazaar.shop and we shall process the refund to you within 30 days from date of 
receipt of complaint. Once we issue your refund, it takes additional time for your financial 
institution to make funds available in your account, which can vary from 2-10 days from 
the date of refund processing. All orders are manually processed on the Website and sent 
for shipment as soon as they are placed. During this process we incur some irreversible 
fees. Therefore, while we understand that orders might need to be changed sometimes, we 
are unable to do it free of charge after a certain point. We strictly adhere to the following 
cancellation policy: 
• If you cancel your order BEFORE it has been shipped, you will not be charged any cancellation 
fee;


## Embedding

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [11]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8868\412152783.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Creating a vector Store

In [12]:
from langchain_community.vectorstores import Chroma

In [15]:
vector_store = Chroma.from_documents(documents = pdf_chunks,embedding = embedding_model, persist_directory = "D:\\GenAI\\Agentic AI\\Test_Projects\\RAG-Project\\vector-store\\chroma_db")

vector_store.persist()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8868\3252197189.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


Lets verify by reloading the vector store

In [16]:
print("Total Vectors in Store: ", vector_store._collection.count())

Total Vectors in Store:  18


# Querying

### Create a retriever

In [20]:
retriever = vector_store.as_retriever(search_kwargs={"k":3})

### Create an LLM

In [17]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

### Build RAG chain

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List
from langchain_core.documents import Document


In [ ]:
#### Helper function to format documents into a string 

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that helps people find information based on the PDF only."),
    ("human", "Context: {context}\n\nQuestion: {question}\n\nAnswer in a concise manner.")
])

In [35]:
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:
query = "What are the cancellation charges?"

In [43]:
response = qa_chain.invoke(query)

print("Response: ", response)

Response:  **Cancellation charges**

- **Before the order is shipped:** No cancellation fee.  
- **After the order is shipped:** The cancellation is handled as a return, and any standard return fees apply (as defined by the seller).  
- **If you cancel 7–15 days after placing the order:** A charge equal to **the number of days %** of the order total is deducted.  
  - *Example:* Cancel 9 days after ordering → 9 % of the order value is charged.  
- **If the order is refused at delivery:** A **100 %** charge of the order value is applied, unless you can prove the product was tampered with, damaged, faulty, or incomplete (provide photographic/documentary evidence).
